Thinking 1: XGBoost生成CART树考虑了树的复杂度，GDBT未考虑；XGBoost是拟合上一轮损失函数的二阶导展开，GDBT是拟合上一轮损失函数的一阶导展开，因此XGBoost的准确性更高，且满足相同的训练效果需要的迭代次数更少；XGboost支持并行计算，GBDT只能串行计算。

Thinking 2: 搭建过三层神经网络，对FashionMNIST数据集作过分类预测。

Thinking 3: 在工作中，需要构建用户画像，其中包含年龄、性别、地域、收入、学历、职业等维度；用户行为特征，包含时间段、频次、时长、收藏、点击、喜欢、评分等维度；用户消费特征，包含消费习惯、购买意向、是否对促销敏感等维度；Item特征，包含分类、评分、生产商、产地等维度。

In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv('C:/Desktop/开课吧/BI_core/L06/voice/voice.csv')
'''
df.head()
df.info()
df.corr()
'''
print('样本数：{}'.format(df.shape[0]))
print('男性个数：{}'.format(df[df.label=='male'].shape[0]))
print('女性个数：{}'.format(df[df.label=='female'].shape[0]))

样本数：3168
男性个数：1584
女性个数：1584


In [5]:
# 分割特征和target
X = df.iloc[:, :-1]
#print(X.head())

# 特征编码
from sklearn.preprocessing import LabelEncoder
gender_enc = LabelEncoder()
y = df.iloc[:,-1]
y = gender_enc.fit_transform(y)
#print(y)

# 数据规范化
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit_transform(X)
#X.head()

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

from sklearn.svm import SVC
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
svc = SVC()
svc.fit(X_train, y_train)
y_pred = svc.predict(X_test)
print('SVM准确率：', metrics.accuracy_score(y_test, y_pred))

svc = SVC(kernel = 'linear')
svc.fit(X_train, y_train)
y_pred = svc.predict(X_test)
print('Linear SVM准确率：', metrics.accuracy_score(y_test, y_pred))

lr = LogisticRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
print('LR准确率：', metrics.accuracy_score(y_test, y_pred))

knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print('KNN准确率：', metrics.accuracy_score(y_test, y_pred))

bayes = GaussianNB()
bayes.fit(X_train, y_train)
y_pred = bayes.predict(X_test)
print('Naive Bayes准确率：', metrics.accuracy_score(y_test, y_pred))

rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
print('Random Forest准确率：', metrics.accuracy_score(y_test, y_pred))

SVM准确率： 0.6876971608832808
Linear SVM准确率： 0.9353312302839116
LR准确率： 0.917981072555205
KNN准确率： 0.7003154574132492
Naive Bayes准确率： 0.8943217665615142
Random Forest准确率： 0.9842271293375394


In [24]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

ada = AdaBoostClassifier()
ada.fit(X_train, y_train)
y_pred = ada.predict(X_test)
print('AdaBoost准确率：', metrics.accuracy_score(y_test, y_pred))

tree = DecisionTreeClassifier()
tree.fit(X_train, y_train)
y_pred = tree.predict(X_test)
print('DecisionTree准确率：', metrics.accuracy_score(y_test, y_pred))

AdaBoost准确率： 0.9716088328075709
DecisionTree准确率： 0.9605678233438486


Top 3: Random Forest, AdaBoost, Decision Tree

In [4]:
from tpot import TPOTClassifier
tpot = TPOTClassifier(generations=5, population_size=20, verbosity=2)
tpot.fit(X_train, y_train)
print(tpot.score(X_test, y_test))
tpot.export('tpot_pipeline.py')

Optimization Progress: 100%|██████████| 120/120 [06:43<00:00,  5.73s/pipeline]
Generation 5 - Current best internal CV score: 0.9798723016114321
Best pipeline: GradientBoostingClassifier(DecisionTreeClassifier(PolynomialFeatures(input_matrix, degree=2, include_bias=False, interaction_only=False), criterion=gini, max_depth=8, min_samples_leaf=15, min_samples_split=6), learning_rate=1.0, max_depth=10, max_features=0.05, min_samples_leaf=14, min_samples_split=8, n_estimators=100, subsample=0.7500000000000001)
0.9889589905362776
